# Census Data Tools

MORPC works regularly with census data, including but not limited to ACS 5 and 1-year, Decennial Census, PEP, and geographies. The following module is useful for gathering and organizing census data for processes in various workflow. Those workflows are linked when appropriate. 

In [ ]:
import morpc

## API functions and variables

api_get() is a low-level wrapper for Census API requests that returns the results as a pandas dataframe. If necessary, it splits the request into several smaller requests to bypass the 50-variable limit imposed by the API.  

The resulting dataframe is indexed by GEOID (regardless of whether it was requested) and omits other fields that are not requested but which are returned automatically with each API request (e.g. "state", "county") 

In [ ]:
url = 'https://api.census.gov/data/2022/acs/acs1'
params = {
    "get": "GEO_ID,NAME,B01001_001E",
    "for": "county:049,041",
    "in": "state:39"
}

In [ ]:
api = morpc.census.api_get(url, params)

In [ ]:
api

## American Community Survey (ACS) Data Class

When using ACS data, generally we will be digesting data produded using the [morpc-censusacs-fetch](https://github.com/morpc/morpc-censusacs-fetch) workflow. The data that is produced from that script is by default saved in its output_data folders ./morpc-censusacs-fetch/output_data/

The Census ACS Fetch script leverages the `acs_data` class form `morpc.census`


### Create an initial object which represents a variable in the ACS data api.

The class takes 3 arguments:

1. variable group number
2. the year
3. the type of survey (1 or 5 year estimates)

In [ ]:
import morpc

In [ ]:
acs = morpc.census.acs_data('B02001', '2023', '5')

The initial call creates queries the Census for the variable definitions and returns a dictionary of the available variables in the group. see `acs.VARS`

In [ ]:
acs.VARS

The initial call alse fetchs a list of dimensions from a cached json file in ./morpc/census/acs_variable_group.json and is stored in morpc.census.ACS_VAR_GROUPS.

#### Manual verfication for variable dimension names. 

The list of dimensions are automatically created from the Census Variable labels and need verified before being used. If the dimesion names have not be verified, the will not be stored. Navigate to the JSON and check to make sure that there are the correct number of dimension and that they are in the correct order. Change the verfication field to `true`.

In [ ]:
acs.DIMENSIONS

### Query the API for the deisred variables and geography

The `.query()` method queries the API and caches the data in memory under `acs.DATA`. At the same time it creates a frictionless schema that corrosponds with the data. 

#### scope:
These are pre-defined sumlevels and scopes for commonly queried geographies. see `morpc.census.SCOPES`.

In [ ]:
morpc.census.SCOPES

In [ ]:
acs = acs.query(scope='region15-counties')

In [ ]:
data = acs.DATA
data.head()

### For custom queries, use for and in parameters to pass to api query. 

#### for_param:
(optional) The geographies for which to call the the query "state:*" represents all states. "state:39" represent Ohio.

#### in_param:
(optional) A filter for the for parameter. In combinations this allows you do call for small geograhpies inside larger ones. 

> Examples: for_param="county:\*", in_param="state:39" would get all counties in Ohio.
> for_param="tract:\*", in_param='state:39,county:041,049' gets all census tracts in Delaware and Franklin Counties.

### Filter the variables using the get parameter

#### get_param:
(Optional) If you want to return a subset of variables, they can be passed here as a list.

### Dimension Tables

When the query is called the class makes table with the dimensions included that can be used to get summaries of the data. 

This can be used to get quick queries for summaries. 

In [ ]:
acs = morpc.census.acs_data('B02001', '2023', '5').query(scope='region15-counties')

In [ ]:
acs.DIM_TABLE.LONG

In [ ]:
acs.DIM_TABLE.WIDE

In [ ]:
acs.DIM_TABLE.WIDE.loc[[('Total', 'Total'), ('White alone', 'Total'), ('Black or African American alone', 'Total')]].T

### Save raw data (not dim table) as a frictionless resource with schema

After querying the data, save the data as a frictionless resource with reasonable descriptors. 

In [ ]:
acs.save(output_dir='./temp_data/')

In [ ]:
acs.SCHEMA

In [ ]:
acs.RESOURCE

## Load data from cached file

In [1]:
import morpc

In [2]:
acs = morpc.census.acs_data('B02001', '2023', '5').load(scope='region15-counties', dirname='./temp_data/')

morpc.frictionless.load_data | INFO | Loading Frictionless Resource file at location morpc-acs5-2023-region15-counties-b02001.resource.yaml
morpc.frictionless.load_data | INFO | Using schema path specified in resource file.
morpc.frictionless.load_data | INFO | Loading data, resource file, and schema (if applicable) from their source locations
morpc.frictionless.load_data | INFO | --> Data file: morpc-acs5-2023-region15-counties-b02001.csv
morpc.frictionless.load_data | INFO | --> Resource file: morpc-acs5-2023-region15-counties-b02001.resource.yaml
morpc.frictionless.load_data | INFO | --> Schema file: morpc-acs5-2023-region15-counties-b02001.schema.yaml
morpc.frictionless.load_data | INFO | Loading data.
morpc.frictionless.load_data | INFO | Loading Frictionless Resource file at location ../../morpc-geos-collect/output_data/morpc-geos.resource.yaml
morpc.frictionless.load_data | INFO | Ignoring schema as directed by useSchema parameter.
morpc.frictionless.load_data | INFO | Loading d

/home/user/Documents/00_repo/MORPC/.venv/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: driver GPKG does not support open option DRIVER


## Georeference the data to map

Add geometries by joining GEOS to DATA.

In [3]:
acs.DATA.join(acs.GEOS)

,NAME,B02001_001E,B02001_001M,B02001_002E,B02001_002M,B02001_003E,B02001_003M,B02001_004E,B02001_004M,B02001_005E,...,B02001_006M,B02001_007E,B02001_007M,B02001_008E,B02001_008M,B02001_009E,B02001_009M,B02001_010E,B02001_010M,geometry
GEO_ID,,,,,,,,,,,,,,,,,,,,,
0500000US39041,"Delaware County, Ohio",221160,-555555555,179940,939,9233,556,117,61,17437,...,89,2057,497,12280,1156,5430,951,6850,784,"POLYGON ((1823116.85 881478.815, 1823475.64 88..."
0500000US39045,"Fairfield County, Ohio",161289,-555555555,133267,635,14324,645,237,164,4173,...,126,1979,452,7162,883,2159,552,5003,686,"POLYGON ((1971222.637 605224.176, 1971196.493 ..."
0500000US39047,"Fayette County, Ohio",28880,-555555555,26377,158,841,195,1,2,108,...,27,197,117,1339,263,477,169,862,195,"POLYGON ((1721811.474 502868.808, 1721795.426 ..."
0500000US39049,"Franklin County, Ohio",1321635,-555555555,799189,2467,304406,3162,3108,739,71315,...,160,40025,2717,103105,3940,38760,3006,64345,3599,"POLYGON ((1769714.921 754470.125, 1769772.783 ..."
0500000US39073,"Hocking County, Ohio",27938,-555555555,26370,377,110,70,9,12,90,...,24,402,292,957,289,306,243,651,109,"POLYGON ((1996131.846 506358.095, 1996002.396 ..."
0500000US39083,"Knox County, Ohio",62888,-555555555,59089,515,590,169,11,15,444,...,4,650,339,2102,508,1050,477,1052,234,"POLYGON ((2055376.301 868263.698, 2055374.748 ..."
0500000US39089,"Licking County, Ohio",180311,-555555555,156892,594,7231,519,39,35,5664,...,46,1499,498,8934,869,2990,573,5944,680,"POLYGON ((1978895.534 817764.245, 1978931.268 ..."
0500000US39091,"Logan County, Ohio",46140,-555555555,42751,201,796,239,45,43,278,...,25,362,146,1888,345,359,181,1529,288,"POLYGON ((1550541.342 907938.451, 1551948.024 ..."
0500000US39097,"Madison County, Ohio",44126,-555555555,38614,187,2080,243,86,54,386,...,12,259,120,2694,353,825,158,1869,311,"POLYGON ((1757251.035 731172.882, 1757187.521 ..."


You can also do the same with dimension tables. 

In [29]:
acs.DIM_TABLE.PERCENT['geometry'] = [acs.GEOS.loc[x, 'geometry'] for x in acs.DIM_TABLE.PERCENT.reset_index()['GEO_ID']]

In [39]:
import geopandas as gpd
gpd.GeoDataFrame(acs.DIM_TABLE.PERCENT.reset_index(), geometry=('geometry',''))

Race                       GEO_ID                    NAME REFERENCE_YEAR  \
Two or More Races                                                          
0                  0500000US39041   Delaware County, Ohio           2023   
1                  0500000US39045  Fairfield County, Ohio           2023   
2                  0500000US39047    Fayette County, Ohio           2023   
3                  0500000US39049   Franklin County, Ohio           2023   
4                  0500000US39073    Hocking County, Ohio           2023   
5                  0500000US39083       Knox County, Ohio           2023   
6                  0500000US39089    Licking County, Ohio           2023   
7                  0500000US39091      Logan County, Ohio           2023   
8                  0500000US39097    Madison County, Ohio           2023   
9                  0500000US39101     Marion County, Ohio           2023   
10                 0500000US39117     Morrow County, Ohio           2023   
11                 0500000US39127      Perry County, Ohio           2023   
12                 0500000US39129   Pickaway County, Ohio           2023   
13                 0500000US39141       Ross County, Ohio           2023   
14                 0500000US39159      Union County, Ohio           2023   

Race              White alone Black or African American alone  \
Two or More Races       Total                           Total   
0                       77.08                            3.96   
1                       79.11                            8.50   
2                       87.29                            2.78   
3                       56.09                           21.37   
4                       91.26                            0.38   
5                       90.92                            0.91   
6                       82.90                            3.82   
7                       89.01                            1.66   
8                       82.47                            4.44   
9                       83.31                            4.95   
10                      92.18                            0.52   
11                      93.11                            0.22   
12                      88.23                            2.87   
13                      85.67                            5.18   
14                      82.46                            2.29   

Race              American Indian and Alaska Native alone Asian alone  \
Two or More Races                                   Total       Total   
0                                                    0.05        7.47   
1                                                    0.14        2.48   
2                                                    0.00        0.36   
3                                                    0.22        5.01   
4                                                    0.03        0.31   
5                                                    0.02        0.68   
6                                                    0.02        2.99   
7                                                    0.09        0.58   
8                                                    0.18        0.82   
9                                                    0.11        0.64   
10                                                   0.09        0.33   
11                                                   0.07        0.11   
12                                                   0.12        0.27   
13                                                   0.15        0.52   
14                                                   0.13        4.60   

Race              Native Hawaiian and Other Pacific Islander alone  \
Two or More Races                                            Total   
0                                                             0.04   
1                                                             0.09   
2                                                             0.06   
3        

## Below should still be functional, but hoping to implement into ACS class

#### Load the data using frictionless.load_data()

In [ ]:
data, resource, schema = morpc.frictionless.load_data('./temp_data/morpc-acs5-2023-state-B01001.resource.yaml', verbose=False)

#### Using ACS_ID_FIELDS to get the fields ids

In [ ]:
morpc.census.acs_generate_universe_table(data.set_index("GEO_ID"), "B01001_001")

#### Create a dimension table with the data and the dimension names

In [ ]:
dim_table = morpc.census.acs_generate_dimension_table(data.set_index("GEO_ID"), schema, idFields=idFields, dimensionNames=["Sex", "Age group"])

In [ ]:
dim_table.loc[dim_table['Variable type'] == 'Estimate'].head()

### Build ACS Variable Group JSON for Dimension names

In [ ]:
import requests
r = requests.get('https://api.census.gov/data/2023/acs/acs5/variables.json')
varjson = r.json()

In [ ]:
groups = {}
for variable in varjson['variables']:
    if variable not in ['for', 'in', 'ucgid', 'GEO_ID', 'AIANHH', 'AIHHTL', 'AIRES', 'ANRC']:
        group = varjson['variables'][variable]['group']
        if not group[-1].isalpha():
            if group not in groups:
                groups[group] = {}
                groups[group]['concept'] = varjson['variables'][variable]['concept'].replace('Year and Over','Year & Over').replace('Indian and Alaska','Indian & Alaska').replace('Hawaiian and Other','Hawaiian & Other')
                groups[group]['dimensions'] = ['TOTAL'] + varjson['variables'][variable]['concept'].replace(' by ',':').replace('Year and Over','Year & Over').replace('Indian and Alaska','Indian & Alaska').replace('Hawaiian and Other','Hawaiian & Other').replace(' and ',':').split(':')
                groups[group]['dimensions_verified'] = False
                variables = {}
                for variable in varjson['variables']:
                    if varjson['variables'][variable]['group'] == group:
                        variables[variable] = varjson['variables'][variable]['label']
                variables = {k: v for k, v in sorted(variables.items(), key=lambda item: item[0])}
                groups[group]['variables'] = variables

In [ ]:
groups = {k: v for k, v in sorted(groups.items(), key=lambda item: item[0])}

In [ ]:
import json
with open('../morpc/census/acs_variable_groups.json', 'w') as file:
    json.dump(groups, file, indent=3)